In [ ]:
pip install pandas

In [2]:
import numpy as np
import pandas as pd
import copy
import math
import heapq

**Get data**

In [3]:
data = pd.read_csv('edge.csv')

In [4]:
data, data.shape

(                                                 WKT     name  description
 0  LINESTRING (106.657734 10.7720657, 106.6581232...   Line 1          NaN
 1  LINESTRING (106.6679937 10.7681858, 106.674394...   Line 3          NaN
 2  LINESTRING (106.6743943 10.7677114, 106.681702...   Line 4          NaN
 3  LINESTRING (106.6581232 10.770494, 106.663574 ...   Line 5          NaN
 4  LINESTRING (106.663574 10.7764293, 106.6655396...   Line 6          NaN
 5  LINESTRING (106.6731007 10.7709472, 106.674394...   Line 7          NaN
 6  LINESTRING (106.657734 10.7720657, 106.6560754...   Line 8          NaN
 7  LINESTRING (106.6589034 10.7806876, 106.663574...   Line 9          NaN
 8  LINESTRING (106.6707618 10.7696614, 106.665539...  Line 10          NaN
 9  LINESTRING (106.6816408 10.7778613, 106.689108...  Line 11          NaN,
 (10, 3))

**HyperParameter**

In [5]:
Points_lst = []
AdjLst = []

**Util Class**

In [6]:
class Point:
    ID = 0
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.ID = Point.ID
        Point.ID += 1

    def getID(self): return self.ID
    def getX(self): return self.x
    def getY(self): return self.y

def EuclideDistance(PointA,PointB):
    distX = PointA.getX() - PointB.getX()
    distY = PointA.getY() - PointB.getY()
    return math.sqrt(distX*distX + distY*distY)

def checkPoint(x,y):
    for idx,point in enumerate(Points_lst):
        if (point.getX() == x) and (point.getY() == y):
           return point.getID()
    return -1

In [7]:
lines = data.shape[0]
for line in range(lines):
    src = data.iloc(0)[line][0]
    src = src.split(' ',1)[1]
    src = src[1: -1]
    src = src.split(', ')
    #print(src)
    List_ID_temp = []
    for point in src:
        point = point.split(' ')
        x,y = float(point[0]),float(point[1])
        
        idx = checkPoint(x,y)
        if idx == -1:
           #Point not in list
           PointTemp = Point(x,y)
           Points_lst.append(PointTemp)
         #   print("-------- \n",AdjLst)
           AdjLst.append([])
         #   print("-------- \n",AdjLst)
           List_ID_temp.append(PointTemp.getID())
        else:
           List_ID_temp.append(idx)
   #  print(List_ID_temp)
    
    lstLen = len(List_ID_temp)

    print(lstLen,List_ID_temp)
    if lstLen >= 2:
       for i in range(lstLen):
          prev = List_ID_temp[(i - 1 + lstLen) % lstLen]
          after = List_ID_temp[(i + 1) % lstLen]
          pres = List_ID_temp[i]
         #  print(prev,after,pres)
         #  print(AdjLst[pres])
          if i != 0 and not (prev in AdjLst[pres]):
             AdjLst[pres].append(prev)
          if i != lstLen - 1 and not (after in AdjLst[pres]):
             AdjLst[pres].append(after)

12 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
5 [4, 12, 8, 13, 10]
3 [12, 14, 9]
3 [1, 15, 3]
4 [15, 16, 17, 7]
2 [6, 12]
5 [0, 18, 19, 20, 17]
2 [19, 15]
2 [5, 16]
3 [7, 21, 13]


In [8]:
AdjLst    

[[1, 18],
 [0, 2, 15],
 [1, 3],
 [2, 4, 15],
 [3, 5, 12],
 [4, 6, 16],
 [5, 7, 12],
 [6, 8, 17, 21],
 [7, 9, 12, 13],
 [8, 10, 14],
 [9, 11, 13],
 [10],
 [4, 8, 14, 6],
 [8, 10, 21],
 [12, 9],
 [1, 3, 16, 19],
 [15, 17, 5],
 [16, 7, 20],
 [0, 19],
 [18, 20, 15],
 [19, 17],
 [7, 13]]

**Dijsktra PriorityQueue**

In [9]:
class SavePath:
    def __init__(self,u):
        self.path = [u]
        self.len = 0
    
    def add(self,v,dist):
        self.path.append(v)
        self.len += dist

    def getPath(self):
        return self.path

    def getLen(self):
        return self.len
    
    def getV(self): 
        return self.path[-1]

In [10]:
class PQ:
    def __init__(self):
        self.pq = []

    def push(self,data,val = None):
        priority = data.getLen()
        heapq.heappush(self.pq,(priority,data))

    def pop(self):
        item = heapq.heappop(self.pq)
        return item

    def isEmpty(self):
        return len(self.pq) == 0

In [11]:
def dijsktraKPath(source,target,k):
    pq = PQ()
    res = []
    pq.push(SavePath(source))

    cnt = np.zeros(len(Points_lst))
    while not pq.isEmpty() and cnt[target] < k:
        top = pq.pop()[1]
        u = top.getV()

        cnt[u] += 1
        if u == target:
           res.append(top)
        if cnt[u] <= k and u != target: #explore
           for v in AdjLst[u]:
              dist = EuclideDistance(Points_lst[u],Points_lst[v])
              if v not in top.getPath():
                 new_route = copy.deepcopy(top)
                 new_route.add(v,dist)
                 pq.push(new_route)
    return res

In [12]:
for path in dijsktraKPath(1,11,8):
    print(path.getPath())

[1, 15, 16, 17, 7, 8, 9, 10, 11]
[1, 15, 16, 17, 7, 8, 13, 10, 11]
[1, 15, 16, 17, 7, 21, 13, 10, 11]
[1, 2, 3, 4, 12, 14, 9, 10, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[1, 2, 3, 4, 12, 8, 9, 10, 11]
[1, 2, 3, 4, 5, 6, 7, 8, 13, 10, 11]
[1, 2, 3, 4, 12, 8, 13, 10, 11]


**Yen algorithm**

In [13]:
def dijkstra(graph, source, target):
    dist = {v: np.inf for v in range(len(graph))}
    dist[source] = 0
    prev = {v: None for v in range(len(graph))}
    visited = set()
    heap = [(0, source)]
    while heap:
        d, u = heapq.heappop(heap)
        if u in visited:
            continue
        visited.add(u)
        if u == target:
            break
        for v in range(len(graph)):
            if graph[u][v] is not None:
                alt = dist[u] + graph[u][v]
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u
                    heapq.heappush(heap, (alt, v))
    path = []
    u = target
    while prev[u] is not None:
        path.append(u)
        u = prev[u]
    path.append(u)
    path.reverse()
    return path, dist[target]

In [14]:
def yen_k_shortest_paths(source, target, k):
    graph = np.zeros((len(AdjLst), len(AdjLst)))
    for u in range(len(AdjLst)):
        for v in range(len(AdjLst)):
            if u == v:
              graph[u][v] = 0
            elif v in AdjLst[u]:
              graph[u][v] = EuclideDistance(Points_lst[u],Points_lst[v])
            else:
              graph[u][v] = None
    A = []
    B = [dijkstra(graph, source, target)]
    for i in range(1, k):
        A.append(B[-1])
        for j in range(len(A[-1][0])-1):
            spur_node = A[-1][0][j]
            root_path = A[-1][0][:j+1]
            edges_removed = []
            for path in A:
                if len(path[0]) > j and root_path == path[0][:j+1]:
                    u = path[0][j]
                    v = path[0][j+1]
                    if graph[u][v] is not None:
                        edges_removed.append((u, v, graph[u][v]))
                        edges_removed.append((v, u, graph[v][u]))
                        graph[u][v] = None
                        graph[v][u] = None
            for node in root_path[:-1]:
                for i in range(len(graph)):
                    if graph[node][i] is not None:
                       edges_removed.append((node, i, graph[node][i]))
                    if graph[i][node] is not None:
                       edges_removed.append((i, node, graph[i][node]))
                    graph[node][i] = None
                    graph[i][node] = None
            spur_path, spur_dist = dijkstra(graph, spur_node, target)
            if spur_path:
                total_path = root_path[:-1] + spur_path
                total_dist = A[-1][1] + spur_dist
                B.append((total_path, total_dist))
            for u, v, w in edges_removed:
                graph[u][v] = w
        if not B:
            break
        B.sort(key=lambda x: x[1])
    return [path for path, dist in sorted(B, key=lambda x: x[1])[:k] if dist != np.inf]

In [15]:
yen_k_shortest_paths(1, 11, 3)

[[1, 15, 16, 17, 7, 8, 9, 10, 11],
 [1, 15, 16, 17, 7, 8, 13, 10, 11],
 [1, 15, 16, 17, 7, 21, 13, 10, 11]]

**Mapping From point to Location**

In [16]:
point_name = pd.read_csv("point.csv")

In [21]:
point_name

,WKT,name,description
0,POINT (106.657734 10.7720657),BKU,NaN
1,POINT (106.6581232 10.770494),Point 1,NaN
2,POINT (106.65999 10.7638855),Point 2,NaN
3,POINT (106.6670945 10.7677021),Point 3,NaN
4,POINT (106.6707618 10.7696614),Point 8,NaN
5,POINT (106.6743943 10.7677114),Point 9,NaN
6,POINT (106.6655396 10.7779067),Point 10,NaN
7,POINT (106.6647332 10.7868058),Point 11,NaN
8,POINT (106.6721096 10.7829016),Point 12,NaN
9,POINT (106.6731007 10.7709472),Point 13,NaN


In [17]:
src_name = "BKU"
target_name = "Dinh"

In [18]:
def getCoord(name):
    data = point_name[point_name['name'] == name]["WKT"].to_list()[0]
    data = data.split(' ',1)[1][1:-1].split(' ')
    return Point(float(data[0]),float(data[1]))

def getClosestPoint(PointTemp):
    minIdx = 0
    minVal = 999999
    for idx,val in enumerate(Points_lst):
        dist = EuclideDistance(PointTemp,val)
        if dist < minVal:
           minIdx = val.getID()
           minVal = dist
    return minIdx

In [19]:
source_idx = getClosestPoint(getCoord(src_name))
target_idx = getClosestPoint(getCoord(target_name))

In [20]:
print(target_idx)
print(source_idx)

11
0


In [21]:
def WaveWater(source_idx,target_idx,k):
    with open('waveWater.txt',mode = 'a') as f:
        for path in dijsktraKPath(source_idx,target_idx,k):
            for idx in path.getPath():
                f.write("{},{}\n".format(Points_lst[idx].getY(),Points_lst[idx].getX()))
            f.write("========================\n")


def YenAlgo(source_idx,target_idx,k):
    with open('yenAlgo.txt',mode = 'a') as f:
        for path in yen_k_shortest_paths(source_idx, target_idx, k):
            for idx in path:
                f.write("{},{}\n".format(Points_lst[idx].getY(),Points_lst[idx].getX()))
            f.write("========================\n")
        

In [22]:
WaveWater(source_idx,target_idx,3)

In [23]:
YenAlgo(source_idx,target_idx,3)